In [12]:
from rllab.algos.trpo import TRPO
from rllab.algos.batch_polopt import BatchSampler
from rllab.baselines.linear_feature_baseline import LinearFeatureBaseline
from rllab.envs.gym_env import GymEnv
from rllab.envs.normalized_env import normalize
from rllab.misc.instrument import run_experiment_lite
from rllab.misc.instrument import VariantGenerator, variant
from rllab.policies.categorical_mlp_policy import CategoricalMLPPolicy
from rllab.policies.gaussian_mlp_policy import GaussianMLPPolicy
from rllab.policies.deterministic_mlp_policy import DeterministicMLPPolicy
from rllab.sampler.utils import collect_data, rollout
from rllab.sampler import parallel_sampler


import gym
import numpy as np
import GPy
import GPyOpt

import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib
import pandas as pd


import plotly
plotly.offline.init_notebook_mode() 
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
import pandas as pd
import copy
import pickle
from datetime import datetime

import random, os
from operator import add, sub

In [8]:
def myf(X):
    X = X.T
    
#     mb = np.zeros((3,1))
#     mb[0,0] = float(0)
#     mb[1,0] = float(9.42477796)*0.6
#     mb[2,0] = float(4.8953899)*0.6
#     env_sim.wrapped_env.env.env.model.body_mass = mb

#     env_sim.wrapped_env.env.env.model.body_mass = env_real.wrapped_env.env.env.model.body_mass
    env_sim_BO.wrapped_env.env.env.model.body_mass = X
        
    dim = env_sim_BO.wrapped_env.env.env.model.data.qpos.shape[0]
#     print("states: ", states)
#     print("actions: ", actions)
#     print("observations: ", observations)    
    env_sim_BO.wrapped_env.env.env.set_state(states[0,:dim],states[0,dim:])
#     env_sim.wrapped_env.env.env.set_state(states[0,:2],states[0,2:])

    sim_states = []
    rewards = []
    env_sim_BO.reset()
    
    for i in range(0,actions.shape[0]):
        next_o, r, d, env_info = env_sim_BO.step(actions[i,:])
        rewards.append(r)        
        sim_states.append(next_o)
#     print(states)
#     print("sim_states: ", sim_states)
    dist = sum(sum(abs(observations[1:,:]-np.array(sim_states))))/observations.shape[0]
#     print("dist: ", dist)
#     print("rewards: ", sum(rewards))
    
    return dist

def BO(bounds, X_init, Y_init):
    print("bounds: ", bounds)
    print("X_init size: ", X_init.size)
    
    global env_sim_BO
    env_sim_BO = normalize(GymEnv(env_name,record_video=False, record_log=False))
    
    if X_init.size == 0:
        myBopt = GPyOpt.methods.BayesianOptimization(myf, domain=bounds)
    else:
        myBopt = GPyOpt.methods.BayesianOptimization(myf, domain=bounds, X = X_init, Y = Y_init)
        
    time_budget = 30
    print("running BO with time budget: ", time_budget)
    myBopt.run_optimization(max_time=time_budget)
    print("myBopt.x_opt: ", myBopt.x_opt, "\n\n\n")
    return myBopt

class VG(VariantGenerator):

    @variant
    def seed(self):
        return list(range(1, 41))
    


In [37]:
############# sim = [0.5,0.9]-[1.1,1.5]* real


global env_name
env_name = "Swimmer-v1"
returns = []
total_iter = 800
test_iter = 10
num_iters = int(total_iter/test_iter)

def run_task(*_):
    # Please note that different environments with different action spaces may
    # require different policies. For example with a Discrete action space, a
    # CategoricalMLPPolicy works, but for a Box action space may need to use
    # a GaussianMLPPolicy (see the trpo_gym_pendulum.py example)
    env_real = normalize(GymEnv(env_name,record_video=False, record_log=False))
    model_body_mass = env_real.wrapped_env.env.env.model.body_mass    
    print("real mass: ", model_body_mass)    
    bounds = []
    for i in range(0,model_body_mass.shape[0]):
        bounds.append({'name': 'mass', 'type': 'continuous', 'domain': (model_body_mass[i,0]*0.5,model_body_mass[i,0]*1.5)})
    
    env_sim = normalize(GymEnv(env_name,record_video=False, record_log=False))
    
    op = random.choice([add, sub])
    factor = op(1, 0.1 + 0.4*np.random.random_sample())
    mb = env_sim.wrapped_env.env.env.model.body_mass
    mb = np.array(mb)
    mb = mb*factor
    env_sim.wrapped_env.env.env.model.body_mass = mb
    policy = GaussianMLPPolicy(
        env_spec=env_sim.spec,
        # The neural network policy should have two hidden layers, each with 32 hidden units.
        hidden_sizes=(32, 32)
    )
    
    X_init = np.empty(shape=(0, 0))
    Y_init = np.empty(shape=(0, 0))
    
    baseline = LinearFeatureBaseline(env_spec=env_sim.spec)
    
    algo = TRPO(
            env=env_sim,
            policy=policy,
            baseline=baseline,
            batch_size=4000,
            max_path_length=1000,
            n_itr=10,
            discount=0.99,
            step_size=0.01,
    #         Uncomment both lines (this and the plot parameter below) to enable plotting
    #         plot=True,
        )
    for i in range(0,num_iters):
        print("iter: ", i)
#         algo.n_itr = algo.n_itr + test_iter
        current_mass = algo.env.wrapped_env.env.env.model.body_mass
        print("algo env mass: ", current_mass)
        algo.train()
        
        ################### Testing in GT parameter #################
                
        env_real = normalize(GymEnv(env_name,record_video=False, record_log=False))
        print("real mass: ", env_real.wrapped_env.env.env.model.body_mass)
        parallel_sampler.populate_task(env_real, algo.policy, scope=algo.scope)
        paths_real = parallel_sampler.sample_paths(
            policy_params=algo.policy.get_param_values(),
            max_samples=algo.batch_size,
            max_path_length=algo.max_path_length,
            env_params=env_real.get_param_values(),
            scope=algo.scope,
        )
        paths_real_returns = [sum(path["rewards"]) for path in paths_real]
        parallel_sampler.terminate_task(scope=algo.scope)
        ave_returns = np.mean(paths_real_returns)
        print("Real sample ave returns: ", ave_returns, "\n\n\n\n")
        returns.append(ave_returns)
        
        env_real = normalize(GymEnv(env_name,record_video=False, record_log=False))
        print("real mass: ", env_real.wrapped_env.env.env.model.body_mass)
        path = collect_data(env_real, algo.policy, max_path_length=1000,
                       animated=False, speedup=1)
        path["returns"] = sum(path["rewards"])
        print("real returns:  ", path['returns'], path['rewards'].shape , "\n\n\n\n")
#         returns.append(path['returns'])
        
        global states, observations, actions
        states = path['states']
        observations = path['observations']
        actions = path['actions']

        print("sim mass: ", algo.env.wrapped_env.env.env.model.body_mass)
        path_2 = rollout(algo.env, algo.policy, max_path_length=1000,
                       animated=False, speedup=1)
        path_2["returns"] = sum(path_2["rewards"])
        print("sim returns: ", path_2['returns'], path_2["rewards"].shape, "\n\n\n\n") 
        
        myBopt = BO(bounds, X_init, Y_init)
        X_init = myBopt.X
        Y_init = myBopt.Y
        
        env_sim_new = normalize(GymEnv(env_name,record_video=False, record_log=False))        
        env_sim_new.wrapped_env.env.env.model.body_mass = np.array(myBopt.x_opt)
        algo = TRPO(
            env=env_sim_new,
            policy=policy,
            baseline=baseline,
            batch_size=4000,
            max_path_length=1000,
            n_itr=10,
            discount=0.99,
            step_size=0.01,
    #         Uncomment both lines (this and the plot parameter below) to enable plotting
    #         plot=True,
        )
        algo.env = env_sim_new        
        ##############################################################
        
    print("returns: ", returns)
    #now = datetime.now()
    #save_data_filename = "BO_returns_"+env_name+"_iter_"+str(total_iter)+"_"+str(now)
    save_data_filename = "./"+mydir+"/BO_returns_"+vv["env"]+"_iter_"+str(vv["seed"])+"_"+datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    with open(save_data_filename, 'wb') as f:
        pickle.dump(returns, f)


In [38]:
variants = VG().variants()
for v in variants:
    print("seed: ", v["seed"])

    if v["seed"]==1:
        global mydir
        mydir = "BO_"+v["env"]+"_"+datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        os.makedirs(mydir)
    run_experiment_lite(
        run_task,
        # Number of parallel workers for sampling
        n_parallel=1,
        # Only keep the snapshot parameters for the last iteration
        snapshot_mode="all",
        # Specifies the seed for the experiment. If this is not provided, a random seed
        # will be used
        seed=v["seed"],
        exp_prefix = "BO_trpo_"+"_32_32_4000_iter_"+str(v["total_iter"])+"_"+mydir,
        variant=v,
    #     plot=True,
    )


seed:  1


KeyError: 'env'

In [35]:
root_path = '../../data/local/BO-trpoSwimmer-v1-32-32-4000-10/swimmer40/'
paths = []
for i in os.listdir(root_path):
    if 'BO' in i:
        paths.append(i)
count = 0

global mydir
mydir = "BO"+env_name+"_"+datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
os.makedirs(mydir)

for path in paths:
    returns = []
    full_path = root_path+path+'/params.pkl'
    data = joblib.load(full_path)
    algo = data['algo']
    for i in range(0,40):
        env_real = normalize(GymEnv(env_name,record_video=False, record_log=False))
        print("real mass: ", env_real.wrapped_env.env.env.model.body_mass)
        parallel_sampler.populate_task(env_real, algo.policy, scope=algo.scope)
        paths_real = parallel_sampler.sample_paths(
            policy_params=algo.policy.get_param_values(),
            max_samples=algo.batch_size,
            max_path_length=algo.max_path_length,
            env_params=env_real.get_param_values(),
            scope=algo.scope,
        )
        paths_real_returns = [sum(path["rewards"]) for path in paths_real]
        parallel_sampler.terminate_task(scope=algo.scope)
        ave_returns = np.mean(paths_real_returns)
        print("Real sample ave returns: ", ave_returns, "\n\n\n\n")
        returns.append(ave_returns)
        
    print("returns: ", returns)
    save_data_filename = "./"+mydir+"/BO_returns_"+env_name+"_iter_"+str(i+40)+"_"+datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    with open(save_data_filename, 'wb') as f:
        pickle.dump(returns, f)

[2017-09-10 16:21:06,591] Making new env: Swimmer-v1


2017-09-10 16:21:06.601565 EDT | observation space: Box(8,)
2017-09-10 16:21:06.602518 EDT | action space: Box(2,)
2017-09-10 16:21:06.760987 EDT | Warning: skipping Gym environment monitoring since snapshot_dir not configured.


[2017-09-10 16:21:06,762] Making new env: Swimmer-v1


2017-09-10 16:21:06.770516 EDT | observation space: Box(8,)
2017-09-10 16:21:06.771720 EDT | action space: Box(2,)
real mass:  [[  0.        ]
 [ 34.55751919]
 [ 34.55751919]
 [ 34.55751919]]
2017-09-10 16:21:06.778284 EDT | Populating workers...
2017-09-10 16:21:06.782069 EDT | Populated


0%                          100%
[##############################] | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00

...


Total time elapsed: 00:00:01
[2017-09-10 16:50:10,778] Making new env: Swimmer-v1


2017-09-10 16:50:10.783713 EDT | observation space: Box(8,)
2017-09-10 16:50:10.784856 EDT | action space: Box(2,)
real mass:  [[  0.        ]
 [ 34.55751919]
 [ 34.55751919]
 [ 34.55751919]]
2017-09-10 16:50:10.788826 EDT | Populating workers...
2017-09-10 16:50:10.790782 EDT | Populated


0%                          100%
[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00

Real sample ave returns:  108.830487043 




returns:  [109.69456028984649, 110.300635668228, 110.52786669525318, 108.78751933437091, 109.88813771479357, 108.39009481540762, 109.24272439340587, 107.83868607767407, 108.42050424406652, 107.94465204099848, 108.47880574838273, 109.78010606032888, 111.25596514970884, 109.13749189818688, 107.15524056748171, 108.75617638839817, 109.50666226069916, 108.48720190283881, 109.1850004624812, 106.53556968447757, 108.4841108442006, 107.7773876378467, 108.54685633951412, 108.88163410501593, 109.96905317260376, 110.41517119185369, 107.33943174079252, 109.28343308411193, 109.15475415303499, 108.8593874288181, 111.47730018281598, 110.92943536096786, 110.76579356204989, 108.22145913586452, 108.13733831504892, 109.88984808168976, 109.81521268094252, 108.3456235849345, 109.44272231141595, 108.83048704261199]



Total time elapsed: 00:00:01


In [29]:
import joblib

data = joblib.load(full_path)

[2017-09-10 15:44:41,453] Making new env: Swimmer-v1


2017-09-10 15:44:41.705345 EDT | observation space: Box(8,)
2017-09-10 15:44:41.706654 EDT | action space: Box(2,)


In [31]:
algo = data['algo']

In [33]:
algo.n_itr

10